# Database Exploration

By **Franklin Oliveira**

-----

This notebook contains some code written to I (Franklin) get accquainted with the `repteis` database. Here you'll find some basic data treatment and adjustments that presented necessary as I started to understand the nature of the information in file <font color='blue'>'Compilacao Livros Repteis - 2 a 10 - 2020_04_28.xls'</font>.

In [1]:
import datetime
import numpy as np
import pandas as pd

from collections import defaultdict

# pacotes para visualização rápida
import seaborn as sns
import matplotlib.pyplot as plt

# pacote para visualização principal
import altair as alt

# habilitando renderizador para notebook
# alt.renderers.enable('notebook')
alt.renderers.enable('default')


# desabilitando limite de linhas
alt.data_transformers.disable_max_rows()

DataTransformerRegistry.enable('default')

## Importing data...

Importing pre-treated data in `1-data_treatment.ipynb`. In this notebook, I'm doing only some minnor adjustments for visualization purposes only. For a full traceback of data treatment, please see the `1-data_treatment` notebook.

In [2]:
NewTable = pd.read_csv('./data/treated_db.csv', sep=';', encoding='utf-8')

/home/franklin/anaconda3/envs/viz/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3062: DtypeWarning: Columns (2) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


<br>

<font size=5>**Color palette per Order**</font>

The image below was used as inspiration (https://color.adobe.com/create/image)

<img src="./src/paleta_cores.jpeg" width='500px'>

Colors: 

- dark_green: #284021
- light_green: #88BF11
- yellow: #D9CB0B
- orange: #D99311
- dark_orange: #BF4417
- light_brown: #BF8D7A

In [3]:
# OBS: caudata is an error and should be removed. 
cores_ordem = {
    'Squamata': '#BF4417',
    'Testudines': '#D9CB0B', 
    'Crocodylia': '#284021'
}

In [4]:
ordens = list(cores_ordem.keys())
cores = list(cores_ordem.values())

<br>

---

## Graphs

### Total amount of catalogations per year

x: Start Year (from Start Date)
y: number of catalogations per year

In [5]:
# counting catalog. per year
teste = NewTable['ano_coleta'].value_counts()
teste = teste.reset_index().rename(columns={'index':'year', 'ano_coleta':'counts'})

In [6]:
# adjusting columns for graphs
teste['year'] = teste['year'].apply(lambda x:str(x).split('/')[0].split('.')[0]).astype(int)
teste = teste.groupby('year').sum().reset_index() # soma do total de bichos coletados por ano

In [7]:
# min e max para eixo X (year)
min_x = teste['year'].min()
max_x = teste['year'].max()

In [8]:
# taking natural log
teste['ln_counts'] = np.log(teste['counts'])

<font color='red'>**ideia:** fazer crescimento em log (referencias em Bio. fazem isso)</font>

In [9]:
temp = alt.Chart(data= teste, width=800, title= 'Natural Logarithm collected animals per year').mark_line().encode(
    x= alt.X('year', type='ordinal', title='Ano de Coleta'),
    y= alt.Y('ln_counts', type='quantitative', title='Contagem')
)

# temp.save('./graphs/log_coletas_por_ano.html')
temp

alt.Chart(...)

-----

## Altitude per family

In [10]:
# subsetting
teste = NewTable[['altitude','familia','ordem','subordem', 'ano_coleta', 'qualificador_atual', 'numero_catalogo', 
                  'genero_atual', 'especie_atual', 'subespecie_atual']].copy()

# sorting
teste = teste.sort_values(['altitude','familia'])

# dropping na
teste.dropna(subset=['altitude'], inplace=True)

# making sure altitude is a floating point number
teste['altitude'] = teste['altitude'].astype(float)

# removing outlier
teste = teste[teste['altitude'] < 7000].copy()

In [12]:
temp = alt.Chart(teste[teste['familia'] != "#n/d"], title='Altitude per family').mark_circle().encode(
    x = alt.X('familia', type='nominal', title='Family', 
              sort= alt.EncodingSortField('altitude', op='max', order='ascending')),
    y = alt.Y('altitude', type='quantitative', title='Altitude (in meters)'),
    color= alt.Color('ordem', scale=alt.Scale(domain=ordens, range=cores)),
    tooltip = alt.Tooltip(['numero_catalogo', 'genero_atual','especie_atual','subespecie_atual', 
                            'qualificador_atual', 'ano_coleta','altitude'])
)

temp.save('./graphs/altitude/altitude_per_family.html')
temp.facet(column='ordem').save('./graphs/altitude/altitude_per_family-facetado.html')


alt_squam = alt.Chart(teste[(teste['ordem'] == 'Squamata') & (teste['familia'] != "#n/d")],
                      title='Altitude per family (Squamata)').mark_circle(color= cores[0]).encode(
    x = alt.X('familia', type='nominal', title='Family', 
              sort= alt.EncodingSortField('altitude', op='max', order='ascending')),
    y = alt.Y('altitude', type='quantitative', title='Altitude (in meters)'),
    tooltip = alt.Tooltip(['numero_catalogo', 'genero_atual','especie_atual','subespecie_atual', 
                           'ordem', 'subordem',
                            'qualificador_atual', 'ano_coleta','altitude'])
)

alt_squam.save('./graphs/altitude/altitude_per_family-squamata.html')
alt_squam

alt.Chart(...)

<br>

## Altitude per genus

In [13]:
teste = NewTable[['altitude','especie_atual','genero_atual','ordem', 'subordem',
                 'familia', 'ano_coleta', 'qualificador_atual', 'numero_catalogo', 'subespecie_atual']].copy()

# making sure altitude is a floating point number
teste['altitude'] = teste['altitude'].astype(float)

# sorting
teste = teste.sort_values(['altitude','genero_atual'])

# dropping na
teste = teste.dropna(subset=['altitude'])

# removing outlier
teste = teste[teste['altitude'] < 7000].copy()

In [14]:
# ordering x-axis per mean altitude - OUTLIER: ordem nula
temp = alt.Chart(teste[~teste['ordem'].isna()], title='Altitude per genus',
                width= 800).mark_circle().encode(
    x = alt.X('genero_atual', type='nominal', title='Genus',
             sort=alt.EncodingSortField('altitude', op="max", order="ascending")),
    y = alt.Y('altitude', type='quantitative', title='Altitude (in meters)'),
    color = alt.Color('ordem', scale= alt.Scale(domain=ordens, range=cores)),
    tooltip = alt.Tooltip(['numero_catalogo', 'genero_atual','especie_atual','subespecie_atual', 
                           'ordem', 'subordem',
                            'qualificador_atual', 'ano_coleta','altitude'])
)

temp.save('./graphs/altitude/genus/altitude-per-genus.html')
temp

alt.Chart(...)

In [15]:
# input do especialista: altitude não faz sentido para crocodilianos
# separating per order
for j in range(2):
    temp = alt.Chart(teste[(~teste['ordem'].isna()) & (teste['ordem'] == ordens[j])], width=800,
                     title=f'Altitude per genus ({ordens[j]})').mark_circle(color= cores[j]).encode(
        x = alt.X('genero_atual', type='nominal', title='Genus',
                 sort=alt.EncodingSortField('altitude', op="max", order="ascending")),
        y = alt.Y('altitude', type='quantitative', title='Altitude (in meters)'),
        tooltip = alt.Tooltip(['numero_catalogo','especie_atual','subespecie_atual', 
                               'ordem', 'subordem',
                            'qualificador_atual', 'ano_coleta','altitude'])
    )

    temp.save(f'./graphs/altitude/genus/altitude_per_genus-{ordens[j]}.html')

temp

alt.Chart(...)

<br>

## Separating per variance groups

**p.s.:** in this case, variance doesn't make much sense for groups with only one or two observations. So I'm separating per "density" (counting) groups as a proxy. 

<font color='red' size=4>Separando grupos de "maior variância" - mais variabilidade ou contagem de pontos para um mesmo gênero/espécie </font>

OBS: acabei separando pela contagem de pontos (generos mais e menos representados)

In [16]:
# input do especialista: altitude faz mais sentido apenas para a ordem Squamata
squamata = teste[teste['ordem'] == 'Squamata'].copy()
squamata['altitude'] = squamata['altitude'].astype(float)

# counting per gender
sort = squamata.groupby('genero_atual').count()['ordem'].reset_index().rename(columns={'ordem':'counts'})

In [17]:
threshold = 2

# mais variabilidade (threshold: counts >=2)
grupo1 = sort[sort['counts'] > threshold]['genero_atual']

# menos variabilidade
grupo2 = sort[sort['counts'] <= threshold]['genero_atual']

#### grupo de maior dispersão

coincide com grupo com mais de 2 pontos

In [18]:
# ordering x-axis per mean altitude
temp = alt.Chart(squamata[(squamata['genero_atual'].isin(grupo1))], width= 800,
                 title='Altitude per genus (Squamata)').mark_circle(color= cores[0]).encode(
    x = alt.X('genero_atual', type='nominal', title='Genus',
             sort=alt.EncodingSortField('altitude', op='max', order="ascending")),
    y = alt.Y('altitude', type='quantitative', title='Altitude (in meters)'),
    tooltip = alt.Tooltip(['numero_catalogo','especie_atual','subespecie_atual', 
                           'ordem', 'subordem',
                            'qualificador_atual', 'ano_coleta','altitude'])
)

temp.save('./graphs/altitude/genus/altitude_per_genus-maior-var.html')
temp

alt.Chart(...)

#### grupo de menor dispersão

menos pontos (generos menos representados)

In [19]:
# não dá para usar como marca gráfica (muitos valores faltando)
teste['qualificador_atual'].value_counts(dropna=False)

NaN     625
sp.      10
cf.       5
aff.      5
c.        1
Name: qualificador_atual, dtype: int64

In [20]:
# ordering x-axis per mean altitude
temp = alt.Chart(squamata[(squamata['genero_atual'].isin(grupo2))], title='Altitude per genus (Squamata)',
                width=800).mark_circle(color= cores[0]).encode(
    x = alt.X('genero_atual', type='nominal', title='Genus',
             sort=alt.EncodingSortField('altitude', op="max", order="ascending")),
    y = alt.Y('altitude', type='quantitative', title='Altitude (in meters)'),
    tooltip = alt.Tooltip(['numero_catalogo','especie_atual','subespecie_atual', 
                           'ordem', 'subordem',
                            'qualificador_atual', 'ano_coleta','altitude'])
)

temp.save('./graphs/altitude/genus/altitude_per_genus-menor-var.html')
temp

alt.Chart(...)

<br>

**Thats it!**

-----